<a href="https://colab.research.google.com/github/hesoyamm/indian_startup_analysis/blob/master/Copy_of_IndianSTARTUP_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ### analyzing the growth of Indian Startups

#### i will try to comment each cell , what i find usefull is revising the older projects when it comes to data science so commenting helps you move things along faster. 
anyone who reads this , i hope it helps



   

In [ ]:
## https://github.com/hesoyamm/indian_startup_analysis/blob/master/Copy_of_IndianSTARTUP_analysis.ipynb

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
##for data visulatization

In [ ]:
##lets select a background style , not important just for representation
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize']=(36,7)

In [ ]:
#for interactivity
import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import interact_manual

## reading the data

In [ ]:
data = pd.read_csv('startup_funding.csv')

In [ ]:
#changing the names of the  columns inside the data 
print(data)

In [ ]:
data.columns =['Sno','Data','StartupName','Industryvertical','SubVertical','City','InvestorSName','InvestmentType','AmountInUSD','Remarks']

In [ ]:
#lets clean the strings
def clean_string(x):
    return str(x).replace('\\xc2\\xa0','').replace('\\\\xc2\\\\xa0','')
## if you dont understand what happended here go to this url:https://stackoverflow.com/questions/10993612/how-to-remove-xa0-from-string-in-python

In [ ]:
#lets apply the function to clean the data
for col in [
'StartupName','Industryvertical','SubVertical','City','InvestorSName','InvestmentType','AmountInUSD','Remarks']:
    data[col] = data[col].apply(lambda x:clean_string(x))
#lambda function is very uselfull for data cleaning purposes so pandas has an function called .apply() in which pandas performs the task for all x . for more infor refer url:https://medium.com/@chaimgluck1/have-messy-text-data-clean-it-with-simple-lambda-functions-645918fcc2fc    

#lets check the head of the data
data.head()

### data cleaning 

In [ ]:
import warnings 
warnings.filterwarnings('ignore') 
## i dont think i need to expalin this.

In [ ]:
#lets calculate the total missing values in the data
total = data.isnull().sum().sort_values(ascending =False)


In [ ]:
#percentage of missing data 
percent = ((data.isnull().sum()/data.isnull().count())*100).sort_values(ascending =False)

In [ ]:
#for representation point of view i will stopre the two values in a dataset called missing_data
missing_data = pd.concat([total,percent],axis=1,keys=['Total','Percent %'])

In [ ]:
#lets check the head of the data
missing_data

In [ ]:
#lets check the values in the remark column
data['Remarks'].value_counts()

In [ ]:
##AS YOU CAN OBSERVE REMARK HAS LOT OP NAN AND HIGH CORNDINAL VALUES
data = data.drop(['Remarks'],axis=1)


In [ ]:
data.columns

In [ ]:
print(data['AmountInUSD'])

In [ ]:
### now this is little bit tricky u need to observe how the amount value is stored in the columnd AmountinUsd
def clean_amount(x):
    x = ''.join([c for c in str(x) if c in ['0','1','2','3','4','5','6','7','8','9']])
    #we are replacing each anomaly with empty string ""
    x=str(x).replace(",","").replace("+","")
    x=str(x).lower().replace("ubdisclosed","")
    x=str(x).lower().replace("n/a","")
    if x == '':
        x = '-9'
    return x
##now the -999 is to label that a startup has not got any funding


In [ ]:
#lets apply the function on the column
data["AmountInUSD"] = data["AmountInUSD"].apply(lambda x:float(clean_amount(x)))
print(data['AmountInUSD'])





## WORD OF ADVICE
*if you dont understand the last two cells you need to read up on DATA CLEANING
## some say Data scientist spend 70 to 80% of there time cleaning data 
so here are some resources to help you out 
### for learning
https://realpython.com/python-data-cleaning-numpy-pandas/
### for practising 
https://analyticsindiamag.com/10-datasets-for-data-cleaning-practice-for-beginners/


In [ ]:
#lets check the head of the column after cleaning it
plt.rcParams['figure.figsize'] = (15,3)
data['AmountInUSD'].plot(kind='line',color ='black')
plt.title('Distribution of Amount',fontsize =15)
plt.show()

## cleaning the dates
---this is hard labour simply put


In [ ]:
print(data['Data'].head())

In [ ]:
data['Data'].dtype
## note 
## When you see dtype('O') inside dataframe this means Pandas string.

In [ ]:
data['Data'][data['Data'] =='12/05.2015']='12/05/2015'
data['Data'][data['Data'] == '13/04.2015']='13/04/2015'
data['Data'][data['Data'] == '15/01.2015'] ='15/01/2015'
data['Data'][data['Data'] == '22/01//2015'] ='22/01/2015'
data['Data'][data['Data'] == '05/072018']='05/07/2018'
data['Data'][data['Data'] == '01/07/015'] ='01/07/2015'
data['Data'][data['Data'] == '\\\\xc2\\\\xa010/7/2015'] ='10/07/2015'
###how did i know which one of them is not right , refer the next code cell if something is not formated correctly it will give the error below 


![image.png](attachment:48796b1d-f040-4922-a373-ea0968f3be0e.png)

# querying the data using visualization
"

### how does the funding ecosystem changes with respect to Time?

In [ ]:
#converting them into a Datetime object
data['yearmonth'] = (pd.to_datetime(data['Data'],
                                   format="%d/%m/%Y").dt.year*100)+(pd.to_datetime(data['Data'],format = "%d/%m/%Y").dt.month)
print(data['yearmonth'])


#### how did i know how to use the above method ?
refer:https://www.kaggle.com/rtatman/data-cleaning-challenge-parsing-dates

In [ ]:
temp=data['yearmonth'].value_counts().sort_values(ascending =False).head(10)

In [ ]:
print("No. of funding per month in decreasing order (TOp 10)\n",temp)

In [ ]:
year_month = data['yearmonth'].value_counts()

In [ ]:
#lets plot the data
plt.rcParams['figure.figsize'] = (15,7)
sns.barplot(year_month.index,year_month.values,palette= 'copper')
plt.xticks(rotation =90)
plt.xlabel('year-month of transaction',fontsize=12)
plt.ylabel('No. of funding made',fontsize=12)
plt.title('year-month Distribution',fontsize=16)
plt.show()

### analyse the above graph
-one the major thing aspiring data scientist ignore is anaylsis part. They are all into understadnign the code which is fine and well but its not what should be the outcome , your outcome should be to learn from the data , that why industry pay so much amount to data visualizer engineers(i mean crazy money).


** in 2015-16 investments were on high rise and since then the funding has only decreased.(that sucks)

In [ ]:
#lets check the maximum funding of a startup
print("maximum funding to a startup is :",data['AmountInUSD'].dropna().sort_values().max())

In [ ]:
#lets check the startips with more than 50crore+ funding
@interact
def check(column = 'AmountInUSD', x= 50000):#50 crore funding of startups
    return data[data[column] > x].sort_values(by = 'AmountInUSD',ascending =False)

In [ ]:
# lets check out different ventures of PATYM
data[data.StartupName=='Paytm']

In [ ]:
#lets check the minimum funding in a startup
print('Minimum funding to a Startup is :',data['AmountInUSD'].dropna().sort_values().min())

In [ ]:
#lets check the startups with least funding 
data[['AmountInUSD','StartupName']].sort_values(by= 'AmountInUSD',ascending=True).head(5)

### analysis time

--- making the startups getting zero funding as a constant helped out sorting least funded startups


---in cell above observe different venture capital investments for paytm over the years

In [ ]:
data[data["AmountInUSD"]==99900.0].shape

In [ ]:
#average funding
print("ON average Indian startup got funding of: ",data['AmountInUSD'].dropna().sort_values().mean())

In [ ]:
#lets check the no. of funding each startups got 
#remeber i told you representation is everything when it comes to data analysis, you are showing to someone for them to believe in your conclusion
#make it trendy 
print("total startups funded:" , len(data["StartupName"].unique()))


In [ ]:
print(data['StartupName'].value_counts().head(20))

##initalizing the variable for splitting purpose.
startupname= data['StartupName'].value_counts().head(20)

In [ ]:
##if you didn't understand what happende in the above code refer this:
y=startupname.index
print(y)
x=startupname.values
print(x)

#### main reason people use data visualization is it removes the mundane out of the equation , the above cell and the below show the same thing but in a different manner.

--swiggy has been funded 8 times , paytm 7 and so on..


In [ ]:
#lets plot the data
plt.rcParams['figure.figsize'] = (12,15)
sns.barplot(y=startupname.index,x=startupname.values,alpha =0.9,palette='Dark2')
#When using seaborn barplot I can specify an alpha for which it makes the bars semi-translucent

plt.xticks(rotation = 'vertical')
plt.xlabel('Startup name',fontsize=12)
plt.ylabel('Number of funding made',fontsize=12)
plt.title('Number of funding Startups have ',fontsize=15)
plt.show()


### seaborn

--right now i am doing all my visualization through seaborn but there are many more packages but this is simplest to use.


--if you dont understand the code for sns(seaborn) refer this :https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Python_Seaborn_Cheat_Sh\eet.pdf

### what kind of industries are more prefered for startups?

In [ ]:
data.columns

In [ ]:
#lets analyse the Industry of the Startups
#data_cleaning
# i  would replace the nan values with Consumer technology. 
# the more defined data we have the bette our model.
data['Industryvertical'] = data['Industryvertical'].replace('nan','Consumer Technology')

In [ ]:
industry = data['Industryvertical'].value_counts().head(10)

In [ ]:
print(industry)
##if want to understand how replacing nan helped ,try to run this cell without replacing 

In [ ]:
#lets plot the data
plt.rcParams['figure.figsize'] = (15,5)
sns.barplot(industry.index,industry.values,palette = 'autumn')
plt.xticks(rotation='vertical')
plt.xlabel('Industry vertical of startups',fontsize=12)
plt.ylabel('Number of funding made',fontsize=12)
plt.title('Industry vertical of startups with number of funding',fontsize =16)
plt.show()

In [ ]:
#lets analyse the sub vertical industries
industry = data['SubVertical'].value_counts().head(20)
print(industry)

In [ ]:
#so i could have done this step before using the above cell ,
#observe the NAN value is present in the first index. 
industry= data['SubVertical'].value_counts()[1:].head(20)
print(industry)

In [ ]:
#lets plot the data
sns.lineplot(industry.index,industry.values,palette= 'summer')
plt.xticks(rotation='vertical')
plt.xlabel('Subvertical of startups',fontsize=14)
plt.ylabel('Number of funding made',fontsize=12)
plt.title('Subvertical of startups with number of funding',fontsize=19)
plt.show()

## does location also play a role in determing th growth of a Startup ?

In [ ]:
#analysing the effect of cities on a Startup
#lets clean the data
data['City'] = data['City'].replace(('Bengaluru','nan'),('Banglore','Banglore'))

In [ ]:
city = data['City'].value_counts().head(15)
print(city)

In [ ]:
#lets plot the data
sns.barplot(city.index,city.values,palette='Wistia')
plt.xticks(rotation='vertical')
plt.xlabel('City location of startups',fontsize=12)
plt.ylabel('Number of funding made',fontsize=12)
plt.title('city location of startups with number of funding',fontsize=16)
plt.show()

In [ ]:
from wordcloud import WordCloud


In [ ]:
names= data['InvestorSName'][~pd.isnull(data['InvestorSName'])]

In [ ]:
wordcloud= WordCloud(max_font_size=50,width =500,height=300,background_color='cyan').generate(''.join(names))

In [ ]:
plt.figure(figsize=(15,6))
plt.imshow(wordcloud)
plt.axis('on')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
#lets analyze the investors on startups
#lets clean the datasets
data['InvestorSName'][data['InvestorSName'] == 'Undisclosed investors'] = 'Undisclosed Investors'
data['InvestorSName'][data['InvestorSName'] == 'undisclosed investors'] ='Undisclosed Investors'
data['InvestorSName'][data["InvestorSName"] == 'undisclosed investor'] = 'Undisclosed Investors'
data['InvestorSName'][data["InvestorSName"] == 'Undisclosed investor'] = 'Undisclosed Investors'
data['InvestorSName'][data['InvestorSName'] == 'Undisclosed Investors'] = 'Undisclosed Investors'
data['InvestorSName'][data['InvestorSName'] == 'Undisclosed'] ='Undisclosed Investors'
data['InvestorSName'][data['InvestorSName'] =='nan'] = 'Undisclosed Investors'
#probably not the best solution for this,you could research more on this. 
#usefull commits are always welcomed

In [ ]:
#lets check the value counts

investors =data['InvestorSName'].value_counts().head(15)
print(investors)

In [ ]:
#lets plot the data 
sns.barplot(investors.index,investors.values,palette='hot')
plt.xticks(rotation='vertical')
plt.xlabel('Investors Names',fontsize=12)
plt.ylabel("No. of funding made",fontsize=12)
plt.title("Invesots Names with number of funding",fontsize=16)
plt.show()

### what are the different types of funding for Startups?

In [ ]:
#lets analyze the investment
investment= data['InvestmentType'].value_counts().head(10)

In [ ]:
print(investment)

In [ ]:
#lets clean the datasets
data['InvestmentType'][data['InvestmentType']=='SeedFunding'] = 'Seed Funding'

In [ ]:
data['InvestmentType'][data['InvestmentType']=='Crowd funding']='Crowd Funding'
data['InvestmentType'][data['InvestmentType']=='PrivateEquity']='Private Equity'
data['InvestmentType'][data['InvestmentType']=='Seed/Angel Funding']='Seed Angel Funding'
data['InvestmentType'][data['InvestmentType']=='Seed/Angel Funding']='Seed Angel Funding'
data['InvestmentType'][data['InvestmentType']=='Seed / Angel Funding']='Seed Angel Funding'
data['InvestmentType'][data['InvestmentType']=='Angel / Seed Funding']='Seed Angel Funding'


In [ ]:
#lets plot the data
sns.barplot(investment.index,investment.values,palette='summer')
plt.xticks(rotation ='vertical')
plt.xlabel('Investment Type',fontsize=12)
plt.ylabel('Number of fundings made',fontsize=12)
plt.title('Investment Type with number of funding',fontsize=16)
plt.show()

##TAKE AWAYS
 1)how EDA can give us good insights into finance data sets.
2)we learned how data visualization are the most significatnt part in providing us with quick clear understanding of the information.
3)Thanks to graphic representation we can virtualize large volumes of data into understandable and coeherent manner which in turn helps us to draw conclusion and insights.
4)DATA CLEANING
5)DATA MANIPULATION 
6)QUERY ANALYSIS
7)DATA VISUALIZATION.
WE USED {BARPLOT,LINEPLOT AND WORD CLOUD}

#DON'T GET OVERWHELMED learn to love struggle.
